In [ ]:
from ast import literal_eval
import boto3
import numpy as np
import sagemaker
from src.mnist_sagemaker_ci_cd.lib.settings import Settings
from sagemaker.serializers import NumpySerializer
from sagemaker.deserializers import JSONDeserializer
from sagemaker.estimator import Estimator
from sagemaker.pytorch import PyTorchModel

In [ ]:
SESSION = sagemaker.Session(boto3.Session(region_name="us-east-1"))
IAM_ROLE = "arn:aws:iam::220582896887:role/programmatic-aws-sagemaker-role-access"
SETTINGS = Settings()

In [ ]:
# This number was grabbed from the MNIST dataset.
sample_number = """[
 [
     [0, 0, 0, 0, 0, 0, 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 84 , 185, 159, 151, 60 , 36 , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 222, 254, 254, 254, 254, 241, 198, 198, 198, 198, 198, 198, 198, 198, 170, 52 , 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 67 , 114, 72 , 114, 163, 227, 254, 225, 254, 254, 254, 250, 229, 254, 254, 140, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0  , 0  , 0  , 0  , 0  , 17 , 66 , 14 , 67 , 67 , 67 , 59 , 21 , 236, 254, 106, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 83 , 253, 209, 18 , 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 22 , 233, 255, 83 , 0  , 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 129, 254, 238, 44 , 0  , 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 59 , 249, 254, 62 , 0  , 0  , 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 133, 254, 187, 5  , 0  , 0  , 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 9  , 205, 248, 58 , 0  , 0  , 0  , 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 126, 254, 182, 0  , 0  , 0  , 0  , 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 75 , 251, 240, 57 , 0  , 0  , 0  , 0  , 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0  , 0  , 0  , 0  , 0  , 0  , 0  , 19 , 221, 254, 166, 0  , 0  , 0  , 0  , 0  , 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0  , 0  , 0  , 0  , 0  , 0  , 3  , 203, 254, 219, 35 , 0  , 0  , 0  , 0  , 0  , 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0  , 0  , 0  , 0  , 0  , 0  , 38 , 254, 254, 77 , 0  , 0  , 0  , 0  , 0  , 0  , 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0  , 0  , 0  , 0  , 0  , 31 , 224, 254, 115, 1  , 0  , 0  , 0  , 0  , 0  , 0  , 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0  , 0  , 0  , 0  , 0  , 133, 254, 254, 52 , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0  , 0  , 0  , 0  , 61 , 242, 254, 254, 52 , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0  , 0  , 0  , 0  , 121, 254, 254, 219, 40 , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0  , 0  , 0  , 0  , 121, 254, 207, 18 , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0  , 0, 0, 0, 0, 0, 0]
 ],
]"""

In [ ]:
data = literal_eval(sample_number)
image = np.array([data], dtype=np.float32)

In [ ]:
model_path = Estimator.attach(
    training_job_name=SETTINGS.github_sha[:7], sagemaker_session=SESSION
).model_data

model = PyTorchModel(
    model_data=model_path,
    role=IAM_ROLE,
    sagemaker_session=SESSION,
    source_dir="./",
    framework_version='2.1',
    py_version='py310',
    entry_point="src/mnist_sagemaker_ci_cd/lib/inference.py",
    name=SETTINGS.github_sha[:7],
    code_location=SETTINGS.output_s3_uri,
)

endpoint_name = f"{SETTINGS.github_sha[:7]}"

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    endpoint_name=endpoint_name,
    # This is required and must match the input type of the inference.py script
    serializer=NumpySerializer(dtype=np.float32),
    deserializer=JSONDeserializer(),
    # End of the deliberate requirement section
    endpoint_logging=True,
)